### Libraries

In [8]:
from jsonapi_client import Session
import pandas as pd
from io import StringIO
import requests
from tqdm import tqdm
import asyncio
import aiohttp
import nest_asyncio

### Functions for asynchronous requests

In [25]:
async def fetch_taxonomy(session, analysis_id):
    url = f"https://www.ebi.ac.uk/metagenomics/api/v1/analyses/{analysis_id}/taxonomy/ssu"
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                df_temp = pd.json_normalize(data["data"])
                df_temp["analysis_id"] = analysis_id
                return df_temp
            else:
                print(f"Error fetching data for {analysis_id}: HTTP {response.status}")
                return None
    except asyncio.TimeoutError:
        print(f"Timeout error fetching taxonomy for {analysis_id}")
        return None

async def fetch_metadata(session, analysis_id):
    url = f"https://www.ebi.ac.uk/metagenomics/api/v1/analyses/{analysis_id}"
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                try:
                    sample_id = data["data"]["relationships"]["sample"]["data"]["id"]
                    sample_url = f"https://www.ebi.ac.uk/metagenomics/api/v1/samples/{sample_id}"
                    async with session.get(sample_url) as sample_response:
                        if sample_response.status == 200:
                            sample_data = await sample_response.json()
                            sample_attributes = sample_data["data"]["attributes"]
                            geographic_location = None
                            for entry in sample_attributes.get("sample-metadata", []):
                                key = entry.get("key", "").lower()
                                if "collection date" in key:
                                    collection_date = entry.get("value")
                                    break
                            return {
                                "analysis_id": analysis_id,
                                "sample_id": sample_id,
                                "sample_name": sample_attributes.get("sample-name"),
                                "collection_date": collection_date,
                                "geographic_location": sample_attributes.get("geo-loc-name"),
                            }
                        else:
                            print(f"Error fetching sample {sample_id}: HTTP {sample_response.status}")
                            return sample_id
                except Exception as e:
                    print(f"Error processing analysis JSON for {analysis_id}: {e}")
                    return None
            else:
                print(f"Error fetching analysis {analysis_id}: HTTP {response.status}")
                return None
    
    except asyncio.TimeoutError:
        print(f"Timeout error fetching metadata for {analysis_id}")
        return None

async def main(analysis_ids):
    timeout = aiohttp.ClientTimeout(total = 600000000)
    conn = aiohttp.TCPConnector(limit = 100000000)
    async with aiohttp.ClientSession(timeout=timeout, connector=conn) as session:
        # Schedule all requests concurrently
        taxonomy_tasks = [fetch_taxonomy(session, aid) for aid in analysis_ids]
        metadata_tasks = [fetch_metadata(session, aid) for aid in analysis_ids]

        # taxonomy_results = await asyncio.gather(*taxonomy_tasks)
        # metadata_results = await asyncio.gather(*metadata_tasks)

        taxonomy_results = []
        for task in tqdm(asyncio.as_completed(taxonomy_tasks), total=len(taxonomy_tasks), desc = "Fetching Taxonomy"):
            result = await task
            taxonomy_results.append(result)

        metadata_results = []
        error_samples_list = []
        for task in tqdm(asyncio.as_completed(metadata_tasks), total = len(metadata_tasks), desc = "Fetching Metadata"):
            result = await task
            if isinstance(result, dict):
                metadata_results.append(result)
            else:
                error_samples_list.append(result)
                print("Added to error list")

        # Filter out any failed (None) results
        taxonomy_dfs = [res for res in taxonomy_results if res is not None]
        metadata_list = [res for res in metadata_results if res is not None]

        taxonomy_df = pd.concat(taxonomy_dfs, ignore_index=True) if taxonomy_dfs else pd.DataFrame()
        metadata_df = pd.DataFrame(metadata_list) if metadata_list else pd.DataFrame()

        return taxonomy_df, metadata_df, error_samples_list

### Global surveillance of antimicrobial resistance (DTU-GE)

In [5]:
# Set study accession
study_accession = "MGYS00001312"

# Create session with MGnify API endpoint
with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
    # Iterate over all analyses in study
    analyses_iter = mgnify.iterate(f"studies/{study_accession}/analyses")
    # Extract JSON from each record
    analyses_json = [record.json for record in analyses_iter]
    # Normalize HSON into pd.DataFrame
    df = pd.json_normalize(analyses_json)

analysis_ids = df["id"]

#### Extract data from analysis

In [ ]:
# dfs = []

# with tqdm(total=len(analysis_ids), desc="Fetching Taxonomy SSU Data", unit="analysis") as pbar:
#     for analysis_id in analysis_ids:

#         url = f"https://www.ebi.ac.uk/metagenomics/api/v1/analyses/{analysis_id}/taxonomy/ssu"
#         response = requests.get(url)

#         if response.status_code == 200:

#             try:
#                 data = response.json()
#                 df_temp = pd.json_normalize(data["data"])
            
#             except Exception as e:
#                 tqdm.write(f"Error processing JSON for {analysis_id}: {e}")
#                 pbar.update(1)
#                 continue

#             df_temp["analysis_id"] = analysis_id
#             dfs.append(df_temp)

#         else:
#             tqdm.write(f"Error fetching data for {analysis_id}: HTTP {response.status_code}")
        
#         pbar.set_postfix_str(f"Remaining: {len(analysis_ids) - pbar.n - 1}")
#         pbar.update(1)

# if dfs:
#     final_df = pd.concat(dfs, ignore_index=True)
#     print("Combined DataFrame:")
#     final_df.head

# else:
#     print("No data was retrieved.")

Fetching Taxonomy SSU Data: 100%|██████████| 413/413 [41:26<00:00,  6.02s/analysis, Remaining: 0]    


Combined DataFrame:


#### Getting metadata from each analysis id sample

In [20]:
metadata_df = []

with tqdm(total=len(analysis_ids), desc="Fetching Analysis Sample Data", unit="analysis") as pbar:
    for analysis_id in analysis_ids:
        url = f"https://www.ebi.ac.uk/metagenomics/api/v1/analyses/{analysis_id}"
        response = requests.get(url)

        if response.status_code == 200:
            try:
                data = response.json()
                sample_id = data["data"]["relationships"]["sample"]["data"]["id"]
                sample_url = f"https://www.ebi.ac.uk/metagenomics/api/v1/samples/{sample_id}"
                sample_response = requests.get(sample_url)

                if sample_response.status_code == 200:
                    try:
                        sample_data = sample_response.json()
                        sample_attributes = sample_data["data"]["attributes"]
                        
                        # Extract geographic location from sample-metadata
                        geographic_location = None
                        sample_metadata_list = sample_attributes.get("sample-metadata", [])
                        
                        # Look for keys containing "geographic location"
                        for entry in sample_metadata_list:
                            key = entry.get("key", "").lower()
                            if "collection date" in key:
                                collection_date = entry.get("value")
                                break  # Stop after first match

                        sample_metadata = {
                            "analysis_id": analysis_id,
                            "sample_id": sample_id,
                            "sample_name": sample_attributes.get("sample-name"),
                            "collection_date": collection_date,
                            "geographic_location": sample_attributes.get("geo-loc-name"),
                        }
                        
                        metadata_df.append(sample_metadata)
                    
                    except Exception as e:
                        tqdm.write(f"Error processing sample JSON for {sample_id}: {e}")
                
                else:
                    tqdm.write(f"Error fetching sample {sample_id}: HTTP {sample_response.status_code}")
            
            except Exception as e:
                tqdm.write(f"Error processing analysis JSON for {analysis_id}: {e}")
        
        else:
            tqdm.write(f"Error fetching analysis {analysis_id}: HTTP {response.status_code}")
        
        pbar.update(1)

final_metadata_df = pd.DataFrame(metadata_df) if metadata_df else pd.DataFrame()
final_metadata_df

Fetching Analysis Sample Data:   1%|          | 5/413 [00:04<06:12,  1.10analysis/s]


KeyboardInterrupt: 

In [21]:
metadata_df

[{'analysis_id': 'MGYA00216443',
  'sample_id': 'ERS1426837',
  'sample_name': 'wastewater metagenome',
  'collection_date': '2016-01-25',
  'geographic_location': 'Slovenia'},
 {'analysis_id': 'MGYA00216444',
  'sample_id': 'ERS1426784',
  'sample_name': 'wastewater metagenome',
  'collection_date': '2016-02-15',
  'geographic_location': 'Brazil'},
 {'analysis_id': 'MGYA00216445',
  'sample_id': 'ERS1426839',
  'sample_name': 'wastewater metagenome',
  'collection_date': '2016-02-11',
  'geographic_location': 'Sweden'},
 {'analysis_id': 'MGYA00216446',
  'sample_id': 'ERS1426784',
  'sample_name': 'wastewater metagenome',
  'collection_date': '2016-02-15',
  'geographic_location': 'Brazil'},
 {'analysis_id': 'MGYA00216447',
  'sample_id': 'ERS1426787',
  'sample_name': 'wastewater metagenome',
  'collection_date': '2016-02-01',
  'geographic_location': 'Canada'}]

#### Asyncio - extraction of data and metadata

In [27]:
# Assuming `df` from your study analysis retrieval code:
analysis_ids = df["id"].tolist()

# Patch the event loop
nest_asyncio.apply()

# Run the asynchronous main function
taxonomy_df, metadata_df,  error_sample_list = asyncio.run(main(analysis_ids))

print("Combined Taxonomy DataFrame:")
print(taxonomy_df.head())
print("\nMetadata DataFrame:")
print(metadata_df.head())

Fetching Metadata:   1%|          | 4/413 [00:39<41:39,  6.11s/it]  

Error fetching sample ERS1443997: HTTP 404
Added to error list


Fetching Metadata:   3%|▎         | 11/413 [00:57<18:04,  2.70s/it]

Error fetching sample ERS1443986: HTTP 404
Added to error list


Fetching Metadata:   7%|▋         | 28/413 [01:27<09:58,  1.55s/it]

...

Error fetching sample ERS1443944: HTTP 404
Added to error list


Fetching Metadata:  99%|█████████▉| 410/413 [06:52<00:01,  2.40it/s]

Error fetching sample ERS1443941: HTTP 404
Added to error list


Fetching Metadata: 100%|██████████| 413/413 [06:53<00:00,  1.00s/it]


Combined Taxonomy DataFrame:
    analysis_id       type                                                 id  \
0  MGYA00216537  organisms             Archaea::Euryarchaeota:Methanomicrobia   
1  MGYA00216537  organisms  Archaea::Euryarchaeota:Methanomicrobia:Methano...   
2  MGYA00216537  organisms                                           Bacteria   
3  MGYA00216537  organisms                      Bacteria:::::::bacterium_LF-3   
4  MGYA00216537  organisms  Bacteria:::::::bacterium_enrichment_culture_cl...   

   attributes.count                                 attributes.lineage  \
0               1.0             Archaea::Euryarchaeota:Methanomicrobia   
1              33.0  Archaea::Euryarchaeota:Methanomicrobia:Methano...   
2             423.0                                           Bacteria   
3               1.0                      Bacteria:::::::bacterium_LF-3   
4               1.0  Bacteria:::::::bacterium_enrichment_culture_cl...   

  attributes.hierarchy.kingdom attribut

#### Define taxonomic rank and creating count data table

In [35]:
rank = "family"

ranked_df = taxonomy_df[(taxonomy_df[f"attributes.hierarchy.{rank}"].notna()) & 
                      (taxonomy_df[f"attributes.hierarchy.{rank}"] != '')
                      ]

grouped_df = ranked_df.groupby(["analysis_id", f"attributes.hierarchy.{rank}"],
                               as_index=False,
                               )["attributes.count"].sum()

wide_df = grouped_df.pivot_table(
    index="analysis_id",
    columns=f"attributes.hierarchy.{rank}",
    values="attributes.count",
    fill_value = 0
).reset_index()

merged_df = metadata_df.merge(
    wide_df,
    on="analysis_id",
    how="left"
).dropna().reset_index(drop=True)

merged_df

,analysis_id,sample_id,sample_name,collection_date,geographic_location,Acanthamoebidae,Acaridae,Acetobacteraceae,Acholeplasmataceae,Acidaminococcaceae,...,Vorticellidae,Vulgatibacteraceae,Wenzhouxiangellaceae,Williamsiaceae,Woeseiaceae,Xanthobacteraceae,Xanthomonadaceae,Yersiniaceae,Zoogloeaceae,Zoothamniidae
0,MGYA00216461,ERS1426787,wastewater metagenome,2016-02-01,Canada,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,2.0,0.0
1,MGYA00216522,ERS1426804,wastewater metagenome,2016-01-25,Georgia,0.0,0.0,0.0,1.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,78.0,0.0,18.0,0.0
2,MGYA00216638,ERS1426851,wastewater metagenome,2016-02-22,USA,0.0,0.0,1.0,0.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,6.0,0.0
3,MGYA00216447,ERS1426787,wastewater metagenome,2016-02-01,Canada,0.0,0.0,19.0,2.0,55.0,...,0.0,0.0,0.0,1.0,0.0,3.0,167.0,0.0,20.0,0.0
4,MGYA00216652,ERS1426821,wastewater metagenome,2016-02-04,Moldova,3.0,0.0,4.0,0.0,115.0,...,0.0,0.0,0.0,0.0,0.0,1.0,163.0,0.0,23.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,MGYA00216634,ERS1426794,wastewater metagenome,2016-02-01,Czech Republic,0.0,0.0,10.0,0.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,120.0,0.0,13.0,0.0
228,MGYA00216625,ERS1426779,wastewater metagenome,2016-02-04,Australia,0.0,0.0,6.0,0.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,103.0,0.0,16.0,0.0
229,MGYA00216668,ERS1426846,wastewater metagenome,2016-02-24,USA,0.0,0.0,18.0,0.0,377.0,...,3.0,1.0,0.0,0.0,0.0,2.0,219.0,1.0,20.0,0.0
230,MGYA00216453,ERS1426849,wastewater metagenome,2016-02-22,USA,0.0,1.0,13.0,16.0,270.0,...,0.0,0.0,0.0,0.0,0.0,3.0,106.0,1.0,74.0,0.0


#### Save .csv file

In [36]:
merged_df.to_csv("datasets/Global_surveillance/MGYS00001312_taxon_family.csv", index = False)